In [35]:
import gc
import os
import talib
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timezone, time


import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from os import getenv
from dotenv import load_dotenv

from strategy import long_short
from config import ccxt_connect, prices_collection, binance_client
from helper import vwap, current_position

load_dotenv()

client = ccxt_connect()
binance = binance_client()
prices = prices_collection()

In [38]:
print('Starting the bot')
print(f"Current time: {datetime.utcnow().strftime('%m-%d-%y-%H:%M:%S')}")

last_date = prices.find().sort('date', -1).limit(1).next()['date']
print(f"Finding the last record in our DB: \n{last_date.strftime('%m-%d-%y-%H:%M:%S')}")

print(f'Querying data from bybit to make current')
new_data_query = binance.get_klines(symbol='ETHUSD', interval=binance.KLINE_INTERVAL_5MINUTE, endTime=int(last_date.timestamp() * 1000))
print('Getting new data from Bybit API (5m interval)')

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Close time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    print(f'Inserting new data into DB')
    prices_collection().insert_many(new_data.to_dict('records'))

df = pd.DataFrame(list(prices.find().sort('date', -1).limit(300)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

print('Calculating signals')
df['signal'] = df.apply(long_short, axis=1)

symbol = 'ETHUSD'

signal = df['signal'].iloc[-1]
price = df['close'].iloc[-1]
position = current_position()
print(f'Current signal: {signal}, current position: {position}')

Starting the bot
Current time: 12-26-22-19:29:49
Finding the last record in our DB: 
12-26-22-17:14:59
Querying data from bybit to make current
Getting new data from Bybit API (5m interval)
Inserting new data into DB
Calculating signals
Current signal: BUY, current position: SELL


In [39]:
df.tail(40)

,_id,open,high,low,close,vol,rsi,macdhist,EMA9,EMA55,EMA200,date,signal
260,63a9cfd28b7245c363620482,1216.49,1216.99,1216.49,1216.99,3.56191,60.460554,0.056139,1216.723856,1216.147237,1217.815917,2022-12-26 16:44:59.999,BUY
261,63a9d0fd8b7245c363620484,1216.86,1217.05,1216.60,1216.60,1.87930,54.654291,0.021375,1216.699085,1216.163407,1217.803819,2022-12-26 16:49:59.999,BUY
262,63a9f6307f44f8a85be4b9ec,1216.88,1217.19,1216.46,1216.62,58.39241,54.893519,-0.003248,1216.683268,1216.179714,1217.792039,2022-12-26 16:49:59.999,None
263,63a9d21028cdadb9e09ab0fa,1216.08,1216.27,1215.92,1215.92,2.22594,45.788400,-0.066166,1216.530614,1216.170439,1217.773412,2022-12-26 16:54:59.999,None
264,63a9f6307f44f8a85be4b9ed,1216.46,1216.64,1215.65,1216.20,40.69693,49.403573,-0.087095,1216.464491,1216.171494,1217.757756,2022-12-26 16:54:59.999,None
265,63a9d34acde3c6672b420661,1215.97,1216.85,1215.97,1216.63,6.33693,54.429496,-0.070839,1216.497593,1216.187870,1217.746535,2022-12-26 16:59:59.999,None
266,63a9f6307f44f8a85be4b9ee,1216.00,1217.07,1216.00,1216.54,60.09983,53.237506,-0.065151,1216.506075,1216.200446,1217.734529,2022-12-26 16:59:59.999,None
267,63a9d7717f44f8a85be4b813,1216.00,1217.07,1216.00,1216.54,60.09983,53.237506,-0.060494,1216.512860,1216.212573,1217.722644,2022-12-26 16:59:59.999,None
268,63a9f6307f44f8a85be4b9ef,1216.49,1216.49,1214.40,1214.40,123.22895,33.192141,-0.193158,1216.090288,1216.147838,1217.689582,2022-12-26 17:04:59.999,SELL
269,63a9d7717f44f8a85be4b814,1216.49,1216.49,1214.40,1214.40,123.22895,33.192141,-0.267929,1215.752230,1216.085415,1217.656850,2022-12-26 17:04:59.999,SELL


In [40]:
orders = client.fetch_closed_orders(symbol)

In [41]:
orders[-1]['info']['side']

'Sell'

In [42]:
int(orders[-1]['info']['qty'])

2681

In [ ]:
signal = 'SELL'

In [ ]:
if signal == 'SELL':
    print(f'Signal went from: {position} to {signal}, flipping to short at {price}')
    balance = client.fetch_balance()
    last_order = client.fetch_closed_orders(symbol)
    if last_order[-1]['info']['side'] == 'Buy':
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = int(last_order[-1]['info']['qty']),
                params = {'reduce_only': True})
    quantity = round((float(balance['ETH']['free']) * .95), 3)
    order = client.create_order(
            symbol = 'ETHUSD',
            type = 'market',
            side = 'sell',
            amount = (price * quantity) * 5,
            params = {'leverage': 5})

In [ ]:
if signal == position or signal == None:
        print('No change in position, sleep for 5min and rescan')

else:
        with open('current_position.txt', 'w') as f:
                f.write(str(signal))

if signal == 'BUY':
        print(f'Signal went from: {position} to {signal}, flipping to long at {price}')
        balance = client.fetch_balance()
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'sell',
                amount = (price * balance['ETH']['used']) * 5,
                params = {'reduce_only': True})
        quantity = round((float(balance['ETH']['free']) * .95), 3)
        order = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = (price * quantity) * 5,
                params = {'leverage': 5})
        print(order)

if signal == 'SELL':
        print(f'Signal went from: {position} to {signal}, flipping to short at {price}')
        balance = client.fetch_balance()
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = (price * balance['ETH']['used']) * 5,
                params = {'reduce_only': True})
        quantity = round((float(balance['ETH']['free']) * .95), 3)
        order = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'sell',
                amount = (price * quantity) * 5,
                params = {'leverage': 5})
        print(order)

In [ ]:
df.to_csv('check.csv', index=False)

In [ ]:
df.plot(y=['close', 'EMA20'], x='date', figsize=(20, 10))